In [1]:
import numpy as np
from pdcdp import *
from dfCoresettree import *
import pandas as pd

In [2]:
N = 1000
K = 3
means = np.array([[0, 0], [5, 5], [10, 0]])
cov = np.eye(2)
# this is a K by N array of random multivariate normal distributions
X = np.vstack([np.random.multivariate_normal(mean, cov, int(N / K)) for mean in means])


In [4]:
df = pd.DataFrame(X, columns=['x', 'y'])

In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import StructType, StructField, StringType


spark = SparkSession \
          .builder \
          .appName("APP") \
          .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/06/18 20:28:12 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [7]:
Tree = dfCoreSetTree(df, 200)
Tree.fit()
coreset = Tree.coreset
NP_coreset = np.array(coreset)
pdcdp = PDCDP(lmbda=7, parts=4)
pdcdp.fit(NP_coreset)
centers = pdcdp.centroids
centers_spark = spark.createDataFrame(centers, ['x', 'y'])
centers_spark.show()

+--------------------+--------------------+
|                   x|                   y|
+--------------------+--------------------+
|   4.972696617449705|   5.041090908514768|
|  10.040616824279715|-0.06219186371192873|
|0.018551735852492424|-0.01069391259366...|
+--------------------+--------------------+

